In [1]:
import pandas as pd
import numpy as np

In [2]:
#load 4 data sets on demographics/stats, crime, income, population
stats_df = pd.read_csv('atlanta_stats_area.csv')
crime_df = pd.read_csv('crime_data.csv')
income_df = pd.read_csv('income_total.csv')
population_df = pd.read_csv('population.csv')

In [3]:
#drop redundant data and NA's and rename columns to be more friendly in stats data frame
stats_df.drop('OBJECTID', 1, inplace=True)
stats_df.drop('POP2010', 1, inplace=True)
stats_df = stats_df.rename(index=str, columns={'NPU':'npu', 'STATISTICA': 'statistical_area', 'NEIGHBORHO': 'neighborhood'})
stats_df.dropna(inplace=True)
stats_df.drop(stats_df.index[59], inplace=True)

In [4]:
#clean up income data frame
income_df.drop('weave_coa_nsa', 1, inplace=True)
income_df.dropna(inplace=True)
income_df = income_df.rename(index=str, columns={'NSA': 'nsa', 'NEIGHBORHOOD': 'neighborhood',
                                                 'Total households': 'total_households',
                                                'Household income less than $35,000': 'income_less_35k',
                                                'Household income $35,000 to $74,999': 'income_35k_75k',
                                                'Household income $75,000 to $200,000': 'income_75k_200k',
                                                'Household income $200,000 or more': 'income_more_200k',
                                                'Median household income': 'median_income',
                                                'Per capita Income': 'per_capita_income'})

In [5]:
#clean up population data frame
population_df.drop('weave_coa_nsa', 1, inplace=True)
population_df.dropna(inplace=True)
population_df = population_df.rename(index=str, columns={'NSA': 'nsa', 'NEIGHBORHOOD': 'neighborhood',
                                                         'Population, 2010': 'population_2010',
                                                         'Population, 2000': 'population_2000',
                                                         'Square Miles': 'sqr_miles',
                                                         'Population Density (per sq mile), 2010': 'population_density'})

In [6]:
#create new column for month crime committed and day crime committed
crime_df['month'] = pd.DatetimeIndex(crime_df['occur_date']).month
crime_df['day'] = pd.DatetimeIndex(crime_df['occur_date']).day

In [7]:
#function multiplies percentage of race by total population, then coverts that to an interger
def pop_total(df, col_1, col_2, new_col):
    df[new_col] = df[col_1] * (df[col_2] / 100)
    df[new_col] = df[new_col].astype(int)

In [8]:
#add population by race
pop_total(stats_df, 'white', 'pop', 'white_total')
pop_total(stats_df, 'black', 'pop', 'black_total')
pop_total(stats_df, 'asian', 'pop', 'asian_total')
pop_total(stats_df, 'hispanic', 'pop', 'hispanic_total')

In [9]:
#function calculates percent income and creates a new column for it
def percent_income(df, col_1, col_2):
    df['%_' + col_2] = (df[col_2] / df[col_1] *100)

percent_income(income_df, 'total_households', 'income_less_35k')
percent_income(income_df, 'total_households', 'income_35k_75k')
percent_income(income_df, 'total_households', 'income_75k_200k')
percent_income(income_df, 'total_households', 'income_more_200k')

In [10]:
#add ratio between median income and per capita income as measure of income inequality
income_df['income_ratio'] = income_df['median_income'] / income_df['per_capita_income']

In [12]:
#calculate the population difference between 2010 and 2000
population_df['pop_diff'] = population_df['population_2010'] - population_df['population_2000']

In [14]:
#calculate population density per square mile
population_df['density_sqrmile'] = population_df['population_density'] / population_df['sqr_miles']
population_df

,nsa,neighborhood,population_2010,population_2000,sqr_miles,population_density,pop_diff,density_sqrmile
0,A01,"Margaret Mitchell, Paces, Pleasant Hill",4061.0,4151.0,4.2,959.0,-90.0,228.333333
1,A02,"Kingswood, Mt. Paran/Northside, Mt. Paran Park...",4207.0,4082.0,4.3,985.0,125.0,229.069767
2,A03,"Chastain Park, Tuxedo Park",3423.0,3085.0,2.8,1209.0,338.0,431.785714
3,B01,Peachtree Heights West,4874.0,4292.0,0.9,5348.0,582.0,5942.222222
4,B02,"Buckhead Forest, South Tuxedo Park",3372.0,3041.0,0.7,4870.0,331.0,6957.142857
5,B03,East Chastain Park,2092.0,1832.0,0.5,3824.0,260.0,7648.000000
6,B04,North Buckhead,8332.0,5653.0,2.7,3124.0,2679.0,1157.037037
7,B05,Brookhaven,2335.0,2144.0,1.0,2354.0,191.0,2354.000000
8,B06,"Buckhead Heights, Lenox, Ridgedale Park",3143.0,2322.0,0.5,6540.0,821.0,13080.000000
9,B07,"Buckhead Village, Peachtree Park",2920.0,1918.0,0.7,4185.0,1002.0,5978.571429
